In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define a function to extract bubble features from an image
def extract_bubble_features(image_path, min_bubble_area=10, max_bubble_area=1000):
    # Load and preprocess the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply bubble segmentation to extract the number of bubbles as a feature
    def segment_bubbles(image):
        # Apply adaptive histogram equalization
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(1, 1))
        equalized_image = clahe.apply(image)

        # Apply adaptive thresholding to segment bubbles
        block_size = 91
        thresholded = cv2.adaptiveThreshold(equalized_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, block_size, 2)

        # Find connected components and filter out small and large regions (bubbles)
        num_labels, labeled_image = cv2.connectedComponents(thresholded)
        bubble_areas = []  # To store areas of bubbles
        for label in range(1, num_labels):
            label_mask = np.uint8(labeled_image == label)
            label_area = np.sum(label_mask)
            if min_bubble_area < label_area < max_bubble_area:
                bubble_areas.append(label_area)
        return len(bubble_areas), sum(bubble_areas), image.shape[0] * image.shape[1]  # Return the number of bubbles, total area, and image area
    
    num_bubbles, total_area, image_area = segment_bubbles(image)
    
    # Calculate the percentage area covered by bubbles
    percentage_area_covered = (total_area / image_area) * 100
    
    # Calculate the uniformity of distribution
    uniformity = num_bubbles / (image_area / 10000)  # Normalize by dividing by 10,000
    
    # Calculate the mean size of bubbles
    mean_bubble_size = total_area / num_bubbles
    
    return num_bubbles, total_area, percentage_area_covered, uniformity, mean_bubble_size

# Specify the path to your dataset folders
positive_class_folder = "./Good_New_4X"
negative_class_folder = "./Bad_New_4X"

# Collect the paths to all images in the dataset
positive_class_images = [os.path.join(positive_class_folder, filename) for filename in os.listdir(positive_class_folder)]
negative_class_images = [os.path.join(negative_class_folder, filename) for filename in os.listdir(negative_class_folder)]

# Create lists to store features and labels
features = []
labels = []

# Extract features and labels from positive class images
for image_path in positive_class_images:
    num_bubbles, total_area, percentage_area_covered, uniformity, mean_bubble_size = extract_bubble_features(image_path)
    features.append([num_bubbles, total_area, percentage_area_covered, uniformity, mean_bubble_size])
    labels.append(1)  # Positive class

# Extract features and labels from negative class images
for image_path in negative_class_images:
    num_bubbles, total_area, percentage_area_covered, uniformity, mean_bubble_size = extract_bubble_features(image_path)
    features.append([num_bubbles, total_area, percentage_area_covered, uniformity, mean_bubble_size])
    labels.append(0)  # Negative class

# Convert the features list to a 2D array
X = np.array(features)

# Save the extracted features and labels to a file
np.save('extracted_features_New_4X.npy', X)
np.save('labels_New_4X.npy', labels)


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

# Load your features and labels
loaded_features = np.load('extracted_features_New_4X.npy')
loaded_labels = np.load('labels_New_4X.npy')

# Initialize the StandardScaler
scaler = StandardScaler()

# Scale the loaded features
loaded_features = scaler.fit_transform(loaded_features)

# Initialize arrays to store the metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_score_list = []
roc_auc_list = []

# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.1, 1]
}

# Initialize the SVM classifier
svm = SVC(probability=True)

# Initialize GridSearchCV
grid_search = GridSearchCV(svm, param_grid, scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))

# Perform GridSearchCV and calculate metrics
grid_search.fit(loaded_features, loaded_labels)

# Get the best estimator from the grid search
best_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:")
print("C:", best_model.C)
print("Kernel:", best_model.kernel)
print("Gamma:", best_model.gamma)

# Stratified Cross-Validation
stratified_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform Stratified Cross-Validation and calculate metrics
for i, (train_index, test_index) in enumerate(stratified_kf.split(loaded_features, loaded_labels)):
    X_train, X_test = loaded_features[train_index], loaded_features[test_index]
    y_train, y_test = loaded_labels[train_index], loaded_labels[test_index]

    # Train the best model on the training data
    best_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]  # Probability of belonging to the positive class

    # Calculate and store accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_list.append(accuracy)

    # Calculate and store precision, recall, F1 score, and ROC-AUC score
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)

    precision_list.append(precision)
    recall_list.append(recall)
    f1_score_list.append(f1)
    roc_auc_list.append(roc_auc)

# Calculate and print average metrics
average_accuracy = np.mean(accuracy_list)
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)
average_f1_score = np.mean(f1_score_list)
average_roc_auc = np.mean(roc_auc_list)

print("Average Accuracy:", average_accuracy)
print("Average Precision:", average_precision)
print("Average Recall:", average_recall)
print("Average F1 Score:", average_f1_score)
print("Average ROC-AUC Score:", average_roc_auc)

# Export metrics to an Excel file
metrics_dict = {
    "Fold": list(range(1, stratified_kf.n_splits + 1)),
    "Accuracy": accuracy_list,
    "Precision": precision_list,
    "Recall": recall_list,
    "F1 Score": f1_score_list,
    "ROC-AUC Score": roc_auc_list,
}

df = pd.DataFrame(metrics_dict)
df.to_excel("svm_metrics_hyperparameter_tuning_new4X.xlsx", index=False)


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE  # Import SMOTE

# Load your features and labels
features_path = './extracted_features_New_4X.npy'
labels_path = './labels_New_4X.npy'

loaded_features = np.load(features_path)
loaded_labels = np.load(labels_path)

# Initialize the StandardScaler
scaler = StandardScaler()

# Scale the loaded features
loaded_features = scaler.fit_transform(loaded_features)

# Initialize the SVM classifier
model = SVC(kernel='sigmoid', probability=True, C = 1, gamma = 1 )

# Initialize arrays to store metrics for each fold
cv_accuracies = []
cv_precisions = []
cv_recalls = []
cv_f1_scores = []
cv_roc_auc_scores = []

# Initialize an array to store the average model coefficients
#average_model_coefficients = np.zeros(loaded_features.shape[1])

# Stratified Cross-Validation
stratified_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform Stratified Cross-Validation and calculate metrics
for fold, (train_index, test_index) in enumerate(stratified_kf.split(loaded_features, loaded_labels), 1):
    X_train, X_test = loaded_features[train_index], loaded_features[test_index]
    y_train, y_test = loaded_labels[train_index], loaded_labels[test_index]

    # Apply SMOTE to balance the training data
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    # Print the number of instances for each class in each fold after SMOTE for training set
    print(f"\nFold {fold} - Training Set (After SMOTE):")
    print("Positive Class Instances:", np.sum(y_train_smote == 1))
    print("Negative Class Instances:", np.sum(y_train_smote == 0))

    # Train the SVM model on the balanced training data
    model.fit(X_train_smote, y_train_smote)

    # Get the coefficients of the support vectors
    #coefficients = model.coef_[0]

    # Accumulate the coefficients for this fold
    #average_model_coefficients += coefficients

    # Evaluate the model on the test set
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Append metrics to the lists
    cv_accuracies.append(accuracy)
    cv_precisions.append(precision)
    cv_recalls.append(recall)
    cv_f1_scores.append(f1)
    cv_roc_auc_scores.append(roc_auc)

    # Print the number of instances for each class in each fold after SMOTE for testing set
    print(f"\nFold {fold} - Testing Set (After SMOTE):")
    print("Positive Class Instances:", np.sum(y_test == 1))
    print("Negative Class Instances:", np.sum(y_test == 0))

# Calculate the average coefficients over all folds
#average_model_coefficients /= stratified_kf.n_splits

# Print the average coefficients
print("\nAverage Model Coefficients:")
#for i, coef in enumerate(average_model_coefficients):
#    print(f"Feature {i+1}: {coef}")

# Print the average metrics over all folds
print("\nAverage Metrics:")
print(f"Average Accuracy: {np.mean(cv_accuracies)}")
print(f"Average Precision: {np.mean(cv_precisions)}")
print(f"Average Recall: {np.mean(cv_recalls)}")
print(f"Average F1 Score: {np.mean(cv_f1_scores)}")
print(f"Average ROC-AUC: {np.mean(cv_roc_auc_scores)}")


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define a function to extract bubble features from an image
def extract_bubble_features(image_path, min_bubble_area=10, max_bubble_area=1000):
    # Load and preprocess the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply bubble segmentation to extract the number of bubbles as a feature
    def segment_bubbles(image):
        # Apply adaptive histogram equalization
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(1, 1))
        equalized_image = clahe.apply(image)

        # Apply adaptive thresholding to segment bubbles
        block_size = 91
        thresholded = cv2.adaptiveThreshold(equalized_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, block_size, 2)

        # Find connected components and filter out small and large regions (bubbles)
        num_labels, labeled_image = cv2.connectedComponents(thresholded)
        bubble_areas = []  # To store areas of bubbles
        for label in range(1, num_labels):
            label_mask = np.uint8(labeled_image == label)
            label_area = np.sum(label_mask)
            if min_bubble_area < label_area < max_bubble_area:
                bubble_areas.append(label_area)
        return len(bubble_areas), sum(bubble_areas), image.shape[0] * image.shape[1]  # Return the number of bubbles, total area, and image area
    
    num_bubbles, total_area, image_area = segment_bubbles(image)
    
    # Calculate the percentage area covered by bubbles
    percentage_area_covered = (total_area / image_area) * 100
    
    # Calculate the uniformity of distribution
    uniformity = num_bubbles / (image_area / 10000)  # Normalize by dividing by 10,000
    
    # Calculate the mean size of bubbles
    mean_bubble_size = total_area / num_bubbles
    
    return num_bubbles, total_area, percentage_area_covered, uniformity, mean_bubble_size

# Specify the path to your dataset folders
positive_class_folder = "./Good_New_10X"
negative_class_folder = "./Bad_New_10X"

# Collect the paths to all images in the dataset
positive_class_images = [os.path.join(positive_class_folder, filename) for filename in os.listdir(positive_class_folder)]
negative_class_images = [os.path.join(negative_class_folder, filename) for filename in os.listdir(negative_class_folder)]

# Create lists to store features and labels
features = []
labels = []

# Extract features and labels from positive class images
for image_path in positive_class_images:
    num_bubbles, total_area, percentage_area_covered, uniformity, mean_bubble_size = extract_bubble_features(image_path)
    features.append([num_bubbles, total_area, percentage_area_covered, uniformity, mean_bubble_size])
    labels.append(1)  # Positive class

# Extract features and labels from negative class images
for image_path in negative_class_images:
    num_bubbles, total_area, percentage_area_covered, uniformity, mean_bubble_size = extract_bubble_features(image_path)
    features.append([num_bubbles, total_area, percentage_area_covered, uniformity, mean_bubble_size])
    labels.append(0)  # Negative class

# Convert the features list to a 2D array
X = np.array(features)

# Save the extracted features and labels to a file
np.save('extracted_features_New_10X.npy', X)
np.save('labels_New_10X.npy', labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# Initialize the SVM classifier
model = SVC(kernel='linear')

# Train the SVM model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

# Load your features and labels
loaded_features = np.load('extracted_features_New_10X.npy')
loaded_labels = np.load('labels_New_10X.npy')

# Initialize the StandardScaler
scaler = StandardScaler()

# Scale the loaded features
loaded_features = scaler.fit_transform(loaded_features)

# Initialize arrays to store the metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_score_list = []
roc_auc_list = []

# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.1, 1]
}

# Initialize the SVM classifier
svm = SVC(probability=True)

# Initialize GridSearchCV
grid_search = GridSearchCV(svm, param_grid, scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))

# Perform GridSearchCV and calculate metrics
grid_search.fit(loaded_features, loaded_labels)

# Get the best estimator from the grid search
best_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:")
print("C:", best_model.C)
print("Kernel:", best_model.kernel)
print("Gamma:", best_model.gamma)

# Stratified Cross-Validation
stratified_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform Stratified Cross-Validation and calculate metrics
for i, (train_index, test_index) in enumerate(stratified_kf.split(loaded_features, loaded_labels)):
    X_train, X_test = loaded_features[train_index], loaded_features[test_index]
    y_train, y_test = loaded_labels[train_index], loaded_labels[test_index]

    # Train the best model on the training data
    best_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]  # Probability of belonging to the positive class

    # Calculate and store accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_list.append(accuracy)

    # Calculate and store precision, recall, F1 score, and ROC-AUC score
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)

    precision_list.append(precision)
    recall_list.append(recall)
    f1_score_list.append(f1)
    roc_auc_list.append(roc_auc)

# Calculate and print average metrics
average_accuracy = np.mean(accuracy_list)
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)
average_f1_score = np.mean(f1_score_list)
average_roc_auc = np.mean(roc_auc_list)

print("Average Accuracy:", average_accuracy)
print("Average Precision:", average_precision)
print("Average Recall:", average_recall)
print("Average F1 Score:", average_f1_score)
print("Average ROC-AUC Score:", average_roc_auc)

# Export metrics to an Excel file
metrics_dict = {
    "Fold": list(range(1, stratified_kf.n_splits + 1)),
    "Accuracy": accuracy_list,
    "Precision": precision_list,
    "Recall": recall_list,
    "F1 Score": f1_score_list,
    "ROC-AUC Score": roc_auc_list,
}

df = pd.DataFrame(metrics_dict)
df.to_excel("svm_metrics_hyperparameter_tuning_new10X.xlsx", index=False)


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE  # Import SMOTE

# Load your features and labels
features_path = './extracted_features_New_10X.npy'
labels_path = './labels_New_10X.npy'

loaded_features = np.load(features_path)
loaded_labels = np.load(labels_path)

# Initialize the StandardScaler
scaler = StandardScaler()

# Scale the loaded features
loaded_features = scaler.fit_transform(loaded_features)

# Initialize the SVM classifier
model = SVC(kernel='rbf', probability=True, C = 10, gamma = 'scale' )

# Initialize arrays to store metrics for each fold
cv_accuracies = []
cv_precisions = []
cv_recalls = []
cv_f1_scores = []
cv_roc_auc_scores = []

# Initialize an array to store the average model coefficients
#average_model_coefficients = np.zeros(loaded_features.shape[1])

# Stratified Cross-Validation
stratified_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform Stratified Cross-Validation and calculate metrics
for fold, (train_index, test_index) in enumerate(stratified_kf.split(loaded_features, loaded_labels), 1):
    X_train, X_test = loaded_features[train_index], loaded_features[test_index]
    y_train, y_test = loaded_labels[train_index], loaded_labels[test_index]

    # Apply SMOTE to balance the training data
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    # Print the number of instances for each class in each fold after SMOTE for training set
    print(f"\nFold {fold} - Training Set (After SMOTE):")
    print("Positive Class Instances:", np.sum(y_train_smote == 1))
    print("Negative Class Instances:", np.sum(y_train_smote == 0))

    # Train the SVM model on the balanced training data
    model.fit(X_train_smote, y_train_smote)

    # Get the coefficients of the support vectors
    #coefficients = model.coef_[0]

    # Accumulate the coefficients for this fold
    #average_model_coefficients += coefficients

    # Evaluate the model on the test set
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Append metrics to the lists
    cv_accuracies.append(accuracy)
    cv_precisions.append(precision)
    cv_recalls.append(recall)
    cv_f1_scores.append(f1)
    cv_roc_auc_scores.append(roc_auc)

    # Print the number of instances for each class in each fold after SMOTE for testing set
    print(f"\nFold {fold} - Testing Set (After SMOTE):")
    print("Positive Class Instances:", np.sum(y_test == 1))
    print("Negative Class Instances:", np.sum(y_test == 0))

# Calculate the average coefficients over all folds
#average_model_coefficients /= stratified_kf.n_splits

# Print the average coefficients
print("\nAverage Model Coefficients:")
#for i, coef in enumerate(average_model_coefficients):
#    print(f"Feature {i+1}: {coef}")

# Print the average metrics over all folds
print("\nAverage Metrics:")
print(f"Average Accuracy: {np.mean(cv_accuracies)}")
print(f"Average Precision: {np.mean(cv_precisions)}")
print(f"Average Recall: {np.mean(cv_recalls)}")
print(f"Average F1 Score: {np.mean(cv_f1_scores)}")
print(f"Average ROC-AUC: {np.mean(cv_roc_auc_scores)}")


In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the extracted features and labels from the saved files
X = np.load('extracted_features_4X_reg.npy')
y = np.load('labels_4X_reg.npy')

# Initialize the Random Forest Regressor model
model = RandomForestRegressor()

# Define the number of splits for stratified cross-validation
n_splits = 5  # You can adjust this based on your preference

# Initialize StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Perform stratified cross-validation
for i, (train_index, test_index) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate metrics for evaluation
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Print metrics for each split
    print(f"\nSplit {i + 1}:")
    print("Mean Absolute Error:", mae)
    print("Mean Squared Error:", mse)
    print("Root Mean Squared Error:", rmse)


In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the extracted features and labels from the saved files
X = np.load('extracted_features_10X_reg.npy')
y = np.load('labels_10X_reg.npy')

# Initialize the Random Forest Regressor model
model = RandomForestRegressor()

# Define the number of splits for stratified cross-validation
n_splits = 5  # You can adjust this based on your preference

# Initialize StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Perform stratified cross-validation
for i, (train_index, test_index) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate metrics for evaluation
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Print metrics for each split
    print(f"\nSplit {i + 1}:")
    print("Mean Absolute Error:", mae)
    print("Mean Squared Error:", mse)
    print("Root Mean Squared Error:", rmse)
